In [1]:
from tensorflow.keras import models

In [2]:
model_n100 = models.load_model('../../models/Nasdaq_100_150k')

In [3]:
model_n100.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_46 (LSTM)               (None, 30, 200)           171200    
_________________________________________________________________
lstm_47 (LSTM)               (None, 200)               320800    
_________________________________________________________________
dropout_69 (Dropout)         (None, 200)               0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 200)               800       
_________________________________________________________________
dense_92 (Dense)             (None, 200)               40200     
_________________________________________________________________
dropout_70 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_93 (Dense)             (None, 100)             

In [4]:
from mlchartist.preprocessing import fit_train_scaler, train_test_split_multiple_companies
from mlchartist.array_builder import full_dataset_randomised_arrays

import pandas as pd
import numpy as np

In [ ]:
test_tickers = [
    PAYX
CERN
MYL
ILMN
ADBE
DLTR
MXIM
ALGN
ADP
KLAC
FAST
AMAT
NTES
NFLX
MELI
]

In [15]:
nasdaq100 = pd.read_csv('../../raw_data/nasdaq100.csv', header=None)
nasdaq100_list = list(nasdaq100.values.flatten())
rand_tickers = random.sample(nasdaq100_list, 100)

joined_df = pd.DataFrame()

# Takes nrows from history from each company from the sample
nrows = 3000

for ticker in rand_tickers:
    ticker_path = '../../raw_data/processed/' + ticker.strip().lower() + '.csv'
    ticker_df = pd.read_csv(ticker_path, nrows=nrows)
    joined_df = joined_df.append(ticker_df)

joined_df 19103

train_set 17603
test_set 1500


In [8]:
joined_df.keys()

Index(['ticker', 'date', 'RSI', 'Stochastic', 'Stochastic_signal', 'ADI',
       'OBV', 'ATR', 'ADX', 'ADX_pos', 'ADX_neg', 'MACD', 'MACD_diff',
       'MACD_signal', '5TD_return', '10TD_return', '20TD_return'],
      dtype='object')

In [10]:
INPUT_COLS = ['RSI', 'Stochastic', 'Stochastic_signal', 'ATR', 'ADX', 'ADX_pos', 'ADX_neg', 'MACD', 'MACD_diff',
              'MACD_signal', '10TD_return']
TARGET_COLS=['10TD_return']
outlier_validation={'5TD_return': [-0.3, 0.3]}

stride = 1


train_x, train_y, test_x, test_y, scaler = full_dataset_randomised_arrays(unsplit_df=joined_df, 
                                                                          #IMPORTANT: need use unsplit_df=df
                                                                            #train_df=train_set, 
                                                                            #test_df=test_set, 
                                                                            stride=stride,
                                                                            #fitted_scaler=scaler,
                                                                            #split_dataframe=True,
                                                                            input_cols=INPUT_COLS, 
                                                                            outlier_threshold=1, 
                                                                            outlier_validation=outlier_validation, 
                                                                            check_train_outliers=True,
                                                                            check_test_outliers=False, 
                                                                            target_col=TARGET_COLS, 
                                                                            time_window=30,
                                                                            test_set_size='2Y')

print('')
print('')
print('### Stats ###')
print('train_x', train_x.shape)
print('train_y', train_y.shape)
print('test_x', test_x.shape)
print('test_y', test_y.shape)
print('scaler', scaler)

Train/Test Split: Splitting unsplit dataframe
Scaler: Fitting scaler on train set
3 Companies in Dataset
Starting AAPL: Company 1 of 3
Starting GOOGL: Company 2 of 3
Starting AMZN: Company 3 of 3
All Companies Completed



### Stats ###
train_x (15865, 30, 11)
train_y (15865, 1)
test_x (2166, 30, 11)
test_y (2166, 1)
scaler RobustScaler()


In [24]:
test_x.shape

(2166, 30, 11)

In [21]:
fitted_scaler=scaler

In [22]:
INPUT_COLS = ['RSI', 'Stochastic', 'Stochastic_signal', 'ADI','OBV', 'ATR', 'ADX', 'ADX_pos', 
            'ADX_neg', 'MACD', 'MACD_diff', 'MACD_signal', '10TD_return']
TARGET_COLS=['10TD_return']
outlier_validation={'5TD_return': [-0.3, 0.3]}

stride = 1


holdout_x, holdout_y, scaler = full_dataset_randomised_arrays(unsplit_df=test_set, 
                                                                            stride=stride,
                                                                            split_dataframe=False, ## <-- HERE, use for holdout
                                                                            input_cols=INPUT_COLS, 
                                                                            fitted_scaler=fitted_scaler,
                                                                            outlier_threshold=1, 
                                                                            outlier_validation=outlier_validation, 
                                                                            check_train_outliers=False,
                                                                            check_test_outliers=False, 
                                                                            target_col=TARGET_COLS, 
                                                                            time_window=30,
                                                                            test_set_size='2Y')

print('')
print('')
print('### Stats ###')
print('holdout_x', holdout_x.shape)
print('holdout_y', holdout_y.shape)
print('scaler', scaler)

Train/Test Split: Not splitting dataframe. (Holdout Data)
Scaler: Using provided fitted_scaler
3 Companies in Dataset
Starting AAPL: Company 1 of 3
Starting GOOGL: Company 2 of 3
Starting AMZN: Company 3 of 3
All Companies Completed



### Stats ###
holdout_x (1413, 30, 13)
holdout_y (1413, 1)
scaler RobustScaler()


In [28]:
print('### Stats ###')
print('holdout_x', holdout_x.shape)
print('holdout_y', holdout_y.shape)
print('scaler', scaler)

holdout_y_bin = (holdout_y > 0.0012).astype(int)

### Stats ###
holdout_x (1413, 30, 13)
holdout_y (1413, 1)
scaler RobustScaler()


In [19]:
model_n100.evaluate(holdout_x, holdout_y_bin)

45/45 [==============================] - 6s 124ms/step - loss: 1.0821 - precision_24: 0.4973 - accuracy: 0.4947


[1.082126498222351, 0.49732619524002075, 0.4946921467781067]

In [27]:
print('### Stats ###')
print('train_x', train_x.shape)
print('train_y', train_y.shape)
print('test_x', test_x.shape)
print('test_y', test_y.shape)
print('scaler', scaler)

test_y_bin = (test_y > 0.0012).astype(int)

### Stats ###
train_x (15865, 30, 11)
train_y (15865, 1)
test_x (2166, 30, 11)
test_y (2166, 1)
scaler RobustScaler()


In [26]:
model_n100.evaluate(test_x, test_y_bin)

InvalidArgumentError:    Specified a list with shape [?,13] from a tensor with shape [32,11]
	 [[{{node TensorArrayUnstack/TensorListFromTensor}}]]
	 [[sequential_23/lstm_46/PartitionedCall]] [Op:__inference_test_function_17617]

Function call stack:
test_function -> test_function -> test_function
